<a href="https://colab.research.google.com/github/sw32-seo/ProJAX/blob/main/Distributed_arrays_and_automatic_parallelization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import functools
from typing import Optional

import numpy as np

import jax
import jax.numpy as jnp

In [2]:
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'

if len(jax.local_devices()) < 8:
    raise Exception("Notebook requires 8 devices to run")

# Intro and a quick example

In [3]:
from jax.experimental import mesh_utils
from jax.sharding import PositionalSharding

In [4]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((8,)))

In [5]:
x = jax.random.normal(jax.random.key(0), (8192, 8192))
y = jax.device_put(x, sharding.reshape(4, 2))
jax.debug.visualize_array_sharding(y)

                        
   CPU 0       CPU 1    
                        
                        
   CPU 2       CPU 3    
                        
                        
   CPU 4       CPU 5    
                        
                        
   CPU 6       CPU 7    
                        

In [6]:
z = jnp.sin(y)
jax.debug.visualize_array_sharding(z)

                        
   CPU 0       CPU 1    
                        
                        
   CPU 2       CPU 3    
                        
                        
   CPU 4       CPU 5    
                        
                        
   CPU 6       CPU 7    
                        

`x` is present on single device

In [7]:
%timeit -n 5 -r 5 jnp.sin(x).block_until_ready()

758 ms ± 168 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


`y` is sharded across 8 devices

In [10]:
%timeit -n 5 -r 5 jnp.sin(y).block_until_ready()

657 ms ± 112 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
